In [22]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,desc,split, when,count

In [23]:
# create a Spark session
spark = SparkSession.builder.appName('Start up assignment solution').getOrCreate()

In [24]:
# load the data 
filepath="Listofstartups.csv"

df = spark.read.csv(filepath, header=True)

# see the dataframe
df.show()

+--------------------+--------------------+--------------------+--------------------+
|   Incubation_Center|     Name_of_startup| Location of company|              Sector|
+--------------------+--------------------+--------------------+--------------------+
|      ABES Ghaziabad|            Suryansh|           New Delhi|              EdTech|
|AIC Banasthali Vi...|Thinkpods Educati...| Satara, Maharashtra|             Ed Tech|
|AIC Banasthali Vi...|Inventiway Soluti...| Mumbai, Maharashtra|             HR Tech|
|AIC Banasthali Vi...|C2M Internet Indi...|Lucknow, Uttar Pr...|         Retail Tech|
|AIC Pinnacle Entr...|            Wastinno|   Pune, Maharashtra|         agriculture|
|AIC Pinnacle Entr...|Diabetico - Rise ...|   Pune, Maharashtra|          Healthcare|
|AIC Pinnacle Entr...|  3DGuru Innovations|   Pune, Maharashtra|             EduTech|
|AIC Pinnacle Entr...|     Gupte Education|   Pune, Maharashtra|    Ed Tech, Defence|
|AIC Pinnacle Entr...|Eldew Digital Pvt...|   Pune, Ma

In [25]:
# 1. Find which sector has the most startups
df_most_startups = df.groupby('Sector').count().orderBy(desc('count'))
df_most_startups.show(1)

+----------+-----+
|    Sector|count|
+----------+-----+
|Healthcare|   34|
+----------+-----+
only showing top 1 row



In [26]:
# 2. Split the Location of company into 2 columns, state and city . If state is not present then keep it as null
df = df.withColumn('splitted', split(col('Location of company'), ','))

# Now 'splitted' is an array column, you can use other functions to get 'state' and 'city'
df = df.withColumn('state', col('splitted')[1])
df = df.withColumn('city', col('splitted')[0])

# Drop the 'splitted' column as it's no longer needed
df = df.drop('splitted')
df.show()

+--------------------+--------------------+--------------------+--------------------+--------------+------------+
|   Incubation_Center|     Name_of_startup| Location of company|              Sector|         state|        city|
+--------------------+--------------------+--------------------+--------------------+--------------+------------+
|      ABES Ghaziabad|            Suryansh|           New Delhi|              EdTech|          NULL|   New Delhi|
|AIC Banasthali Vi...|Thinkpods Educati...| Satara, Maharashtra|             Ed Tech|   Maharashtra|      Satara|
|AIC Banasthali Vi...|Inventiway Soluti...| Mumbai, Maharashtra|             HR Tech|   Maharashtra|      Mumbai|
|AIC Banasthali Vi...|C2M Internet Indi...|Lucknow, Uttar Pr...|         Retail Tech| Uttar Pradesh|     Lucknow|
|AIC Pinnacle Entr...|            Wastinno|   Pune, Maharashtra|         agriculture|   Maharashtra|        Pune|
|AIC Pinnacle Entr...|Diabetico - Rise ...|   Pune, Maharashtra|          Healthcare|   

In [27]:
# 3. If Location of company column has a data DIAT,Pune then set state as Maharashtra and city as DIAT Pune.
df = df.withColumn("state", when(col('Location of company')=="DIAT,Pune","Maharashtra").otherwise(col('state')))
df = df.withColumn("city", when(col('Location of company')=="DIAT,Pune","DIAT Pune").otherwise(col('city')))
df.show()

+--------------------+--------------------+--------------------+--------------------+--------------+------------+
|   Incubation_Center|     Name_of_startup| Location of company|              Sector|         state|        city|
+--------------------+--------------------+--------------------+--------------------+--------------+------------+
|      ABES Ghaziabad|            Suryansh|           New Delhi|              EdTech|          NULL|   New Delhi|
|AIC Banasthali Vi...|Thinkpods Educati...| Satara, Maharashtra|             Ed Tech|   Maharashtra|      Satara|
|AIC Banasthali Vi...|Inventiway Soluti...| Mumbai, Maharashtra|             HR Tech|   Maharashtra|      Mumbai|
|AIC Banasthali Vi...|C2M Internet Indi...|Lucknow, Uttar Pr...|         Retail Tech| Uttar Pradesh|     Lucknow|
|AIC Pinnacle Entr...|            Wastinno|   Pune, Maharashtra|         agriculture|   Maharashtra|        Pune|
|AIC Pinnacle Entr...|Diabetico - Rise ...|   Pune, Maharashtra|          Healthcare|   

In [37]:
# 4. If Location of company column has a data Ulhasnagar then set state as Maharashtra and city as Ulhasnagar
df = df.withColumn("state", when(col('Location of company')=="Ulhasnagar","Maharashtra").otherwise(col('state')))
df = df.withColumn("city", when(col('Location of company')=="Ulhasnagar","Ulhasnagar").otherwise(col('city')))
df.show()

+--------------------+--------------------+--------------------+--------------------+--------------+------------+
|   Incubation_Center|     Name_of_startup| Location of company|              Sector|         state|        city|
+--------------------+--------------------+--------------------+--------------------+--------------+------------+
|      ABES Ghaziabad|            Suryansh|           New Delhi|              EdTech|          NULL|   New Delhi|
|AIC Banasthali Vi...|Thinkpods Educati...| Satara, Maharashtra|             Ed Tech|   Maharashtra|      Satara|
|AIC Banasthali Vi...|Inventiway Soluti...| Mumbai, Maharashtra|             HR Tech|   Maharashtra|      Mumbai|
|AIC Banasthali Vi...|C2M Internet Indi...|Lucknow, Uttar Pr...|         Retail Tech| Uttar Pradesh|     Lucknow|
|AIC Pinnacle Entr...|            Wastinno|   Pune, Maharashtra|         agriculture|   Maharashtra|        Pune|
|AIC Pinnacle Entr...|Diabetico - Rise ...|   Pune, Maharashtra|          Healthcare|   

In [40]:
# 5. Find which State has the max number of startups
df_state_max = df.groupby('state').count().orderBy(desc('count')).show()

+---------------+-----+
|          state|count|
+---------------+-----+
|           NULL|   66|
|      Karnataka|   35|
|    Maharashtra|   32|
|      Tamilnadu|   24|
|  Uttar Pradesh|   24|
|      Telangana|   12|
|          Delhi|    9|
|      Rajasthan|    7|
|        Gujarat|    7|
|     Tamil Nadu|    6|
|        Haryana|    5|
| Andhra Pradesh|    4|
|    West Bengal|    2|
|    Maharashtra|    2|
|             HP|    1|
| Andhra Pradesh|    1|
|    IIT Roorkee|    1|
|      Hyderabad|    1|
|          BIHAR|    1|
|         Punjab|    1|
+---------------+-----+



In [41]:
df.show()

+--------------------+--------------------+--------------------+--------------------+--------------+------------+
|   Incubation_Center|     Name_of_startup| Location of company|              Sector|         state|        city|
+--------------------+--------------------+--------------------+--------------------+--------------+------------+
|      ABES Ghaziabad|            Suryansh|           New Delhi|              EdTech|          NULL|   New Delhi|
|AIC Banasthali Vi...|Thinkpods Educati...| Satara, Maharashtra|             Ed Tech|   Maharashtra|      Satara|
|AIC Banasthali Vi...|Inventiway Soluti...| Mumbai, Maharashtra|             HR Tech|   Maharashtra|      Mumbai|
|AIC Banasthali Vi...|C2M Internet Indi...|Lucknow, Uttar Pr...|         Retail Tech| Uttar Pradesh|     Lucknow|
|AIC Pinnacle Entr...|            Wastinno|   Pune, Maharashtra|         agriculture|   Maharashtra|        Pune|
|AIC Pinnacle Entr...|Diabetico - Rise ...|   Pune, Maharashtra|          Healthcare|   

In [49]:
# 6. Find all the startups from Maharashtra .
df_mh = df.filter(col('state')==' Maharashtra')
df_mh.show() 

+--------------------+--------------------+--------------------+--------------------+------------+-------+
|   Incubation_Center|     Name_of_startup| Location of company|              Sector|       state|   city|
+--------------------+--------------------+--------------------+--------------------+------------+-------+
|AIC Banasthali Vi...|Thinkpods Educati...| Satara, Maharashtra|             Ed Tech| Maharashtra| Satara|
|AIC Banasthali Vi...|Inventiway Soluti...| Mumbai, Maharashtra|             HR Tech| Maharashtra| Mumbai|
|AIC Pinnacle Entr...|            Wastinno|   Pune, Maharashtra|         agriculture| Maharashtra|   Pune|
|AIC Pinnacle Entr...|Diabetico - Rise ...|   Pune, Maharashtra|          Healthcare| Maharashtra|   Pune|
|AIC Pinnacle Entr...|  3DGuru Innovations|   Pune, Maharashtra|             EduTech| Maharashtra|   Pune|
|AIC Pinnacle Entr...|     Gupte Education|   Pune, Maharashtra|    Ed Tech, Defence| Maharashtra|   Pune|
|AIC Pinnacle Entr...|Eldew Digital P

In [36]:
# 7. How many startups were formed in Healthcare sector
df_health_startups = df.filter(col('Sector')=="Healthcare").count()
print('Total number of startups in Healthcare sector : ',df_health_startups)

Total number of startups in Healthcare sector :  34


In [45]:
# 8. Display all startups from Pune and Nashik
startup_pune_nashik = df.filter((col('city')=='Pune') | (col('city')=='Nashik')).show()

+--------------------+--------------------+-------------------+--------------------+------------+----+
|   Incubation_Center|     Name_of_startup|Location of company|              Sector|       state|city|
+--------------------+--------------------+-------------------+--------------------+------------+----+
|AIC Pinnacle Entr...|            Wastinno|  Pune, Maharashtra|         agriculture| Maharashtra|Pune|
|AIC Pinnacle Entr...|Diabetico - Rise ...|  Pune, Maharashtra|          Healthcare| Maharashtra|Pune|
|AIC Pinnacle Entr...|  3DGuru Innovations|  Pune, Maharashtra|             EduTech| Maharashtra|Pune|
|AIC Pinnacle Entr...|     Gupte Education|  Pune, Maharashtra|    Ed Tech, Defence| Maharashtra|Pune|
|AIC Pinnacle Entr...|Eldew Digital Pvt...|  Pune, Maharashtra|  IT, Virtual Events| Maharashtra|Pune|
|AIC Pinnacle Entr...|Secumatic Technol...|  Pune, Maharashtra|             Defense| Maharashtra|Pune|
|AIC Pinnacle Entr...|Catalystgreen Pri...|  Pune, Maharashtra|          

In [48]:
df.filter(col('state')==' Maharashtra').show()

+--------------------+--------------------+--------------------+--------------------+------------+-------+
|   Incubation_Center|     Name_of_startup| Location of company|              Sector|       state|   city|
+--------------------+--------------------+--------------------+--------------------+------------+-------+
|AIC Banasthali Vi...|Thinkpods Educati...| Satara, Maharashtra|             Ed Tech| Maharashtra| Satara|
|AIC Banasthali Vi...|Inventiway Soluti...| Mumbai, Maharashtra|             HR Tech| Maharashtra| Mumbai|
|AIC Pinnacle Entr...|            Wastinno|   Pune, Maharashtra|         agriculture| Maharashtra|   Pune|
|AIC Pinnacle Entr...|Diabetico - Rise ...|   Pune, Maharashtra|          Healthcare| Maharashtra|   Pune|
|AIC Pinnacle Entr...|  3DGuru Innovations|   Pune, Maharashtra|             EduTech| Maharashtra|   Pune|
|AIC Pinnacle Entr...|     Gupte Education|   Pune, Maharashtra|    Ed Tech, Defence| Maharashtra|   Pune|
|AIC Pinnacle Entr...|Eldew Digital P

In [50]:
# 9. Sort the cities in Maharashtra in descending order of the count of startups
df_mh1= df.filter(col('state')==' Maharashtra').groupBy('city').count().orderBy(desc('count')).show()

+-------+-----+
|   city|count|
+-------+-----+
|   Pune|   13|
| Mumbai|    9|
|Nashik |    5|
| Satara|    2|
| Nagpur|    2|
|  Thane|    1|
+-------+-----+



In [58]:
# 10. How many startups are in South India. That is states Karnataka ,Tamilnadu , Telangana , Andhra Pradesh
south_list=[' Karnataka' ,' Tamilnadu' , ' Telangana' , ' Andhra Pradesh']
df_south = df.filter(col('state').isin(south_list)).count()
print(df_south)

75


In [60]:
# 11. How many startups are in Gujarat
df_gujju = df.filter(col('state')==' Gujarat').count()
print(df_gujju)

7


In [64]:
# 12. How many startups are in North India.That is states other than Karnataka , Tamilnadu , Telangana , Andhra Pradesh and Maharashtra
df_north = df.filter(~col('state').isin(south_list + [' Maharashtra'])).count()
print(df_north)

68


In [66]:
# 13. What is the percentage of startup initiative from South India and Maharashtra
total_startups = df.count()
df_sm = df.filter(col('state').isin(south_list + [' Maharashtra'])).count()
df_sm_pct = (df_sm/total_startups) * 100
print('The percentage of startup initiative from South India and Maharashtra : ',df_sm_pct)

The percentage of startup initiative from South India and Maharashtra :  44.398340248962654


In [68]:
# 14. What is the percentage contribution of startup from Maharashtra
df_mhc  = df.filter(col('state')==' Maharashtra').count()
df_mh_pct = (df_mhc/total_startups) * 100
print(df_mh_pct)

13.278008298755188


In [69]:
# 15. What is the percentage contribution of startup from Gujarat
df_mhc  = df.filter(col('state')==' Gujarat').count()
df_mh_pct = (df_mhc/total_startups) * 100
print(df_mh_pct)

2.904564315352697


In [70]:
# 16. Replace state with null values to Unknown
df = df.withColumn("state", when(col('state').isNull(), "Unknown").otherwise(col('state'))).show()

+--------------------+--------------------+--------------------+--------------------+--------------+------------+
|   Incubation_Center|     Name_of_startup| Location of company|              Sector|         state|        city|
+--------------------+--------------------+--------------------+--------------------+--------------+------------+
|      ABES Ghaziabad|            Suryansh|           New Delhi|              EdTech|       Unknown|   New Delhi|
|AIC Banasthali Vi...|Thinkpods Educati...| Satara, Maharashtra|             Ed Tech|   Maharashtra|      Satara|
|AIC Banasthali Vi...|Inventiway Soluti...| Mumbai, Maharashtra|             HR Tech|   Maharashtra|      Mumbai|
|AIC Banasthali Vi...|C2M Internet Indi...|Lucknow, Uttar Pr...|         Retail Tech| Uttar Pradesh|     Lucknow|
|AIC Pinnacle Entr...|            Wastinno|   Pune, Maharashtra|         agriculture|   Maharashtra|        Pune|
|AIC Pinnacle Entr...|Diabetico - Rise ...|   Pune, Maharashtra|          Healthcare|   

In [ ]:
# 17. Store the DataFrame with following Schema into a Hive table
# StartUps_Spark with partitioning done on columns state and
# city
# root
# |-- Incubation_Center: string (nullable = false)
# |-- Name_of_startup: string (nullable = false)
# |-- Location_of_company: string (nullable = false)
# |-- Sector: string (nullable = false)
# |-- city: string (nullable = false)
# |-- state: string (nullable = false)
# Hint - Use partitionBy and saveAsTable API from pyspark
df.write.partitionBy('state','city').saveAsTable('startup_table')